In [1]:
#Preparazione ambiente
import os
import sys

#Installazione YOLO (Ultralytics)
!pip install ultralytics

#Clonazione repo da github
!git clone https://github.com/leonardoCosta02/Hawk-AI-CV-Project.git

repo_name = "Hawk-AI-CV-Project"
%cd $repo_name

#Aggiunta percorso al sistema
sys.path.insert(0, os.getcwd())

print("Ambiente pronto")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 79.6 MB/s eta 0:00:00
Cloning into 'Hawk-AI-CV-Project'...
remote: Enumerating objects: 864, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 864 (delta 42), reused 0 (delta 0), pack-reused 775 (from 2)
Receiving objects: 100% (864/864), 87.53 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (372/372), done.
/content/Hawk-AI-CV-Project
Ambiente pronto


In [2]:
#Object Detection con YOLO
from ultralytics import YOLO
import cv2 as cv
import numpy as np
from collections import deque

class ObDetection:
    def __init__(self, model_ver='yolov8x.pt'):
        print(f"Inizializzazione 'Object Detection' ...")
        try:
            self.model = YOLO(model_ver)
        except:
            self.model = YOLO('yolov8m.pt')
        self.trajectory = []

        #Serve a capire se un oggetto si muove o è fermo
        self.back_sub = cv.createBackgroundSubtractorMOG2(history=100, varThreshold=40, detectShadows=False)
        self.avg_area = None

    def applica_clahe(self, frame):
        lab = cv.cvtColor(frame, cv.COLOR_BGR2LAB)
        l, a, b = cv.split(lab)
        clahe = cv.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        limg = cv.merge((cl, a, b))
        return cv.cvtColor(limg, cv.COLOR_LAB2BGR)

    def processa_video(self, video_input, video_output=None, save_video=False):
        cap = cv.VideoCapture(video_input)
        if not cap.isOpened():
            print(f"Errore file: {video_input}")
            return []

        width  = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
        fps    = cap.get(cv.CAP_PROP_FPS)

        out = None
        if save_video and video_output:
            out = cv.VideoWriter(video_output, cv.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

        frame_id = 0
        self.trajectory = []
        self.avg_area = None

        last_u, last_v = None, None
        velocity_u, velocity_v = 0, 0
        recent_velocities_u = deque(maxlen=5) #Media più lunga per stabilità
        recent_velocities_v = deque(maxlen=5)
        missed_frames = 0


        MAX_MISSING = 40        #Non predice troppo a lungo se non serve
        SEARCH_RADIUS = 800
        CONF_THRESHOLD = 0.01
        FRICTION = 0.99
        GRAVITY = 0.4           #La palla cade verso il basso (pixel/frame)
        RESET_AFTER = 10        #Reset rapido se sbaglia

        print(f"Analisi avviata su: {video_input}")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            #Mappa del movimento
            #Immagine in bianco e nero; BIANCO = Si muove, NERO = Fermo
            motion_mask = self.back_sub.apply(frame)

            #CLAHE+YOLO
            frame_enhanced = self.applica_clahe(frame)
            results = self.model(frame_enhanced, verbose=False, classes=[32], conf=CONF_THRESHOLD, imgsz=1280)

            best_candidate = None
            best_candidate_area = 0
            min_dist = float('inf')

            #Reset rapido
            if missed_frames > RESET_AFTER:
                last_u = None
                self.avg_area = None
                recent_velocities_u.clear()
                recent_velocities_v.clear()

            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    w, h = x2-x1, y2-y1
                    area = w * h
                    u, v = (x1 + x2) / 2, (y1 + y2) / 2
                    conf = float(box.conf)

                    #Filtro 1: Movimento Check
                    y1_int, y2_int = int(y1), int(y2)
                    x1_int, x2_int = int(x1), int(x2)

                    #Protezione bordi
                    y1_int, x1_int = max(0, y1_int), max(0, x1_int)
                    y2_int, x2_int = min(height, y2_int), min(width, x2_int)

                    roi_motion = motion_mask[y1_int:y2_int, x1_int:x2_int]
                    if roi_motion.size > 0:
                        motion_ratio = cv.countNonZero(roi_motion) / roi_motion.size
                        #Se meno del 10% dei pixel si muove= ogg statico
                        if motion_ratio < 0.10:
                            continue

                    #Filtro 2: Dimensioni
                    if area < 5 or area > 2000:
                      continue

                    if self.avg_area is not None:
                        ratio = area / self.avg_area
                        if ratio > 3.0 or ratio < 0.33:
                          continue

                    #Distanza
                    dist = float('inf')
                    if last_u is not None:
                        dist = np.sqrt((u - last_u)**2 + (v - last_v)**2)

                    #Selezione
                    is_compatible = False
                    if last_u is not None and dist < SEARCH_RADIUS: is_compatible = True
                    elif conf > 0.4: is_compatible = True
                    elif last_u is None and (best_candidate is None or conf > 0.1): is_compatible = True

                    if is_compatible:
                        if last_u is not None:
                             if dist < min_dist:
                                min_dist = dist
                                best_candidate = (u, v)
                                best_candidate_area = area
                        else:
                             best_candidate = (u, v)
                             best_candidate_area = area

            final_u, final_v = None, None


            if best_candidate:
                #Metto pallino verde
                curr_u, curr_v = best_candidate
                final_u, final_v = curr_u, curr_v

                if self.avg_area is None: self.avg_area = best_candidate_area
                else: self.avg_area = 0.8 * self.avg_area + 0.2 * best_candidate_area

                if last_u is not None:
                    inst_vel_u = curr_u - last_u
                    inst_vel_v = curr_v - last_v
                    recent_velocities_u.append(inst_vel_u)
                    recent_velocities_v.append(inst_vel_v)
                    if len(recent_velocities_u) > 0:
                        velocity_u = sum(recent_velocities_u) / len(recent_velocities_u)
                        velocity_v = sum(recent_velocities_v) / len(recent_velocities_v)

                last_u, last_v = curr_u, curr_v
                missed_frames = 0
                if out: cv.circle(frame, (int(final_u), int(final_v)), 6, (0, 255, 0), -1)

            elif last_u is not None and missed_frames < MAX_MISSING:
                #Pallino rosso grazie a predizione con gravità
                velocity_u *= FRICTION
                velocity_v *= FRICTION

                #Applica Gravità
                velocity_v += GRAVITY

                pred_u = last_u + velocity_u
                pred_v = last_v + velocity_v

                if 0 <= pred_u <= width and 0 <= pred_v <= height:
                    final_u, final_v = pred_u, pred_v
                    last_u, last_v = pred_u, pred_v
                    missed_frames += 1
                    if out:
                        cv.circle(frame, (int(final_u), int(final_v)), 8, (0, 0, 255), -1)
                        cv.putText(frame, "PRED", (int(final_u)+10, int(final_v)),
                                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

            if final_u:
                self.trajectory.append({'frame': frame_id, 'u': final_u, 'v': final_v})

            if out: out.write(frame)
            frame_id += 1
            if frame_id % 50 == 0: print(f"   ... frame {frame_id}")

        cap.release()
        if out: out.release()
        print(f"Analisi completata.")
        return self.trajectory

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# ==========================================
# CELLA ESECUZIONE E GENERAZIONE CSV
# ==========================================
from google.colab import files
import os
import pandas as pd  # <--- Serve per creare il CSV

# 1. MODIFICA QUI IL NOME DEL FILE
# Assicurati che il file 'video2.mp4' sia stato caricato a sinistra
lista_video = ["video2.mp4"]

print(f"Avvio test su {len(lista_video)} video...\n")

# Reinizializziamo la classe
membro2 = ObDetection()

for nome_video in lista_video:
    path_input = None
    if os.path.exists(nome_video): path_input = nome_video
    elif os.path.exists(f"/content/{nome_video}"): path_input = f"/content/{nome_video}"

    if not path_input:
        print(f"❌ Video {nome_video} non trovato. Caricalo nella barra a sinistra!")
        continue

    path_output = f"OUTPUT_{nome_video}"
    print(f"Elaborazione: {nome_video}...")

    # 2. CATTURIAMO LA TRAIETTORIA (Aggiunto 'traiettoria = ...')
    traiettoria = membro2.processa_video(path_input, video_output=path_output, save_video=True)

    # 3. SALVATAGGIO CSV (Questa parte mancava!)
    # Creiamo un nome compatibile con il Giudice: DATI_video2.csv
    nome_base = os.path.splitext(nome_video)[0] # Toglie .mp4
    nome_csv = f"DATI_{nome_base}.csv"

    if len(traiettoria) > 0:
        print(f"💾 Salvataggio dati in: {nome_csv}")
        df = pd.read_json(pd.Series(traiettoria).to_json(orient='values')) # Conversione rapida dict->df
        # Oppure più semplice:
        df = pd.DataFrame(traiettoria)

        df.to_csv(nome_csv, index=False)
        files.download(nome_csv) # Scarica il CSV
    else:
        print("⚠️ Nessuna traiettoria rilevata, niente CSV.")

    # Scarica il video elaborato
    try:
        print(f"Scarico video output: {path_output}")
        files.download(path_output)
    except:
        print(f"   (Scarica manualmente i file dalla barra a sinistra)")

    print("-" * 40)
print("PROGRAMMA TERMINATO")

Avvio test su 1 video...

Inizializzazione 'Object Detection' ...
Elaborazione: video2.mp4...
Analisi avviata su: video2.mp4
   ... frame 50
   ... frame 100
   ... frame 150
   ... frame 200
   ... frame 250
Analisi completata.
💾 Salvataggio dati in: DATI_video2.csv


/tmp/ipython-input-1973231022.py:39: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(pd.Series(traiettoria).to_json(orient='values')) # Conversione rapida dict->df


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Scarico video output: OUTPUT_video2.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

----------------------------------------
PROGRAMMA TERMINATO
